In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline, UNet2DConditionModel, EulerDiscreteScheduler
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file
import uuid
import textwrap
import matplotlib.pyplot as plt


base = "stabilityai/stable-diffusion-xl-base-1.0"
repo = "ByteDance/SDXL-Lightning"
ckpt = "sdxl_lightning_8step_unet.safetensors"

unet = UNet2DConditionModel.from_config(base, subfolder="unet").to("cuda", torch.float16)
unet.load_state_dict(load_file(hf_hub_download(repo, ckpt), device="cuda"))
pipe = StableDiffusionXLPipeline.from_pretrained(base, unet=unet, torch_dtype=torch.float16, variant="fp16").to("cuda")
pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config, timestep_spacing="trailing")

def run_and_show(prompts):
    images_per_row = 4
    total_images = len(prompts)
    rows_needed = (total_images + images_per_row - 1) // images_per_row

    fig_width = 24
    fig_height_per_row = 6
    plt.figure(figsize=(fig_width, rows_needed * fig_height_per_row))

    for i, prompt in enumerate(prompts):
        image = pipe(prompt, num_inference_steps=8, guidance_scale=0).images[0]
        plt.subplot(rows_needed, images_per_row, i+1)
        plt.imshow(image)
        wrapped_text = textwrap.fill(prompt, width=50)
        plt.title(wrapped_text, fontsize=10, pad=10)
        plt.axis('off')

    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.show()

In [ ]:
# generate replicas of famous paintings to see how the model behaves

prompts = [
"A painting of The Starry Night by Vincent van Gogh",
"A painting of The Great Wave off Kanagawa by Katsushika Hokusai",
"A painting of The Scream by Edvard Munch",
"A painting of The Persistence of Memory by Salvador Dalí",
"A painting of The Birth of Venus by Sandro Botticelli",
"A painting of The Kiss by Gustav Klimt",
"A painting of A Sunday Afternoon on the Island of La Grande Jatte by Georges Seurat",
"A painting of The Creation of Adam by Michelangelo",
"A painting of The Christ of Saint John of the Cross by Salvador Dalí",
"A painting of The Christ And The Storm by Rembrandt",
"A painting of Impression, Sunrise by Claude Monet",
"A painting of Daydream or Reverie by Alphonse Mucha",
"A painting of Jean-michel Basquiat by Richard Day",
"A painting of Bal du moulin de la Galette by Pierre-Auguste Renoir",
"A painting of Under the Tree of Life by Gustav Klimt",
"A painting of Man in Bowler Hat by René Magritte",
"A painting of The Autumn by Alphonse Mucha",
"A painting of Blue Lovers by Marc Chagall",
"A painting of Merilyn Monroe by Andy Warhol",
"A painting of Guernica by Pablo Picasso",
"A painting of Dora Maar au Chat by Pablo Picasso",
"A painting of A Girl with Pearl Earring by Johannes Vermeer",
"A painting of Cafe Terrace at Night by Vincent van Gogh",
]*100

run_and_show(prompts)